# Formação Cientista de Dados
### Projeto com Feedback 4
### Prevendo Customer Churn em Operadoras de Telecom

### Gabriel Quaiotti - Abr 2020

Customer Churn (ou Rotatividade de Clientes, em uma tradução livre) refere-se a uma decisão tomada pelo cliente sobre o término do relacionamento comercial. Refere-se também à perda de clientes. A fidelidade do cliente e a rotatividade de clientes sempre somam 100%. Se uma empresa tem uma taxa de fidelidade de 60%, então a taxa de perda de clientes é de 40%. De acordo com a
regra de lucratividade do cliente 80/20, 20% dos clientes estão gerando 80% da receita. Portanto, é muito importante prever os usuários que provavelmente abandonarão o relacionamento comercial e os fatores que afetam as decisões do cliente.

Neste projeto, você deve prever o Customer Churn em uma Operadora de Telecom.

Os datasets de treino e de teste serão fornecidos para você em anexo a este projeto. Seu trabalho é criar um modelo de aprendizagem de máquina que possa prever se um cliente pode ou não cancelar seu plano e qual a probabilidade
de isso ocorrer. O cabeçalho do dataset é uma descrição do tipo de informação em cada coluna.

Usando linguagem Python, recomendamos você criar um modelo de Regressão Logística, para extrair a informação se um cliente vai cancelar seu plano (Sim ou Não) e a probabilidade de uma opção ou outra.

In [1]:
# Libraries
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType

from pyspark.sql.functions import col

In [2]:
# Spark Session - usado para trabalhar com o Spark
spSession = SparkSession.builder.master("local").appName("DSA-TELECOM-BALANCE").getOrCreate()

# BALANCE

In [3]:
# Read train dataset
transformed_rdd = sc.textFile('../dataset/df_transformed.csv')

In [4]:
transformed_rdd.take(5)

['id,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn,state_id,area_code_id',
 '1,128.0,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,9.0,2.7,1.0,0,17.0,0.0',
 '2,107.0,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,9.0,3.7,1.0,0,5.0,0.0',
 '3,137.0,0,0,0,243.4,114,41.38,121.2,110,10.3,162.6,104,7.32,12.2,25.0,3.29,0.0,0,19.0,0.0',
 '4,84.0,1,0,0,299.4,71,50.9,61.9,88,5.26,196.9,89,8.86,6.6,49.0,1.78,4.0,0,5.0,2.0']

In [5]:
# Remove header and split by ','
header = transformed_rdd.first()
transformed_rdd2 = transformed_rdd.filter(lambda line: line != header).map(lambda line: line.split(","))

In [6]:
transformed_rdd2.take(5)

[['1',
  '128.0',
  '0',
  '1',
  '25',
  '265.1',
  '110',
  '45.07',
  '197.4',
  '99',
  '16.78',
  '244.7',
  '91',
  '11.01',
  '10.0',
  '9.0',
  '2.7',
  '1.0',
  '0',
  '17.0',
  '0.0'],
 ['2',
  '107.0',
  '0',
  '1',
  '26',
  '161.6',
  '123',
  '27.47',
  '195.5',
  '103',
  '16.62',
  '254.4',
  '103',
  '11.45',
  '13.7',
  '9.0',
  '3.7',
  '1.0',
  '0',
  '5.0',
  '0.0'],
 ['3',
  '137.0',
  '0',
  '0',
  '0',
  '243.4',
  '114',
  '41.38',
  '121.2',
  '110',
  '10.3',
  '162.6',
  '104',
  '7.32',
  '12.2',
  '25.0',
  '3.29',
  '0.0',
  '0',
  '19.0',
  '0.0'],
 ['4',
  '84.0',
  '1',
  '0',
  '0',
  '299.4',
  '71',
  '50.9',
  '61.9',
  '88',
  '5.26',
  '196.9',
  '89',
  '8.86',
  '6.6',
  '49.0',
  '1.78',
  '4.0',
  '0',
  '5.0',
  '2.0'],
 ['5',
  '75.0',
  '1',
  '0',
  '0',
  '166.7',
  '113',
  '28.34',
  '148.3',
  '122',
  '12.61',
  '186.9',
  '121',
  '8.41',
  '10.1',
  '9.0',
  '2.73',
  '9.0',
  '0',
  '34.0',
  '0.0']]

In [7]:
# Define the dataFrame columns
transformed_fields = [StructField("id", StringType(), True), 
     StructField("account_length", StringType(), True),
     StructField("international_plan", StringType(), True),
     StructField("voice_mail_plan", StringType(), True),
     StructField("number_vmail_messages", StringType(), True),
     StructField("total_day_minutes", StringType(), True),
     StructField("total_day_calls", StringType(), True),
     StructField("total_day_charge", StringType(), True),
     StructField("total_eve_minutes", StringType(), True),
     StructField("total_eve_calls", StringType(), True),
     StructField("total_eve_charge", StringType(), True),
     StructField("total_night_minutes", StringType(), True),
     StructField("total_night_calls", StringType(), True),
     StructField("total_night_charge", StringType(), True),
     StructField("total_intl_minutes", StringType(), True),
     StructField("total_intl_calls", StringType(), True),
     StructField("total_intl_charge", StringType(), True),
     StructField("number_customer_service_calls", StringType(), True),
     StructField("churn", StringType(), True),
     StructField("state_id", StringType(), True),
     StructField("area_code_id", StringType(), True)]

In [8]:
# Define the dataFrame schema
transformed_schema = StructType( transformed_fields )

In [9]:
# Create dataFrame
transformed_ds = spSession.createDataFrame(transformed_rdd2, transformed_schema)

In [10]:
transformed_ds.toPandas().head()

,id,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,...,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn,state_id,area_code_id
0,1,128.0,0,1,25,265.1,110,45.07,197.4,99,...,244.7,91,11.01,10.0,9.0,2.7,1.0,0,17.0,0.0
1,2,107.0,0,1,26,161.6,123,27.47,195.5,103,...,254.4,103,11.45,13.7,9.0,3.7,1.0,0,5.0,0.0
2,3,137.0,0,0,0,243.4,114,41.38,121.2,110,...,162.6,104,7.32,12.2,25.0,3.29,0.0,0,19.0,0.0
3,4,84.0,1,0,0,299.4,71,50.9,61.9,88,...,196.9,89,8.86,6.6,49.0,1.78,4.0,0,5.0,2.0
4,5,75.0,1,0,0,166.7,113,28.34,148.3,122,...,186.9,121,8.41,10.1,9.0,2.73,9.0,0,34.0,0.0


In [11]:
transformed_ds.columns

['id',
 'account_length',
 'international_plan',
 'voice_mail_plan',
 'number_vmail_messages',
 'total_day_minutes',
 'total_day_calls',
 'total_day_charge',
 'total_eve_minutes',
 'total_eve_calls',
 'total_eve_charge',
 'total_night_minutes',
 'total_night_calls',
 'total_night_charge',
 'total_intl_minutes',
 'total_intl_calls',
 'total_intl_charge',
 'number_customer_service_calls',
 'churn',
 'state_id',
 'area_code_id']

In [12]:
# Cast columns type
transformed_ds = transformed_ds.withColumn('id', col('id').cast(IntegerType()))
transformed_ds = transformed_ds.withColumn('account_length', col('account_length').cast(DoubleType()))
transformed_ds = transformed_ds.withColumn('international_plan', col('international_plan').cast(IntegerType()))
transformed_ds = transformed_ds.withColumn('voice_mail_plan', col('voice_mail_plan').cast(IntegerType()))
transformed_ds = transformed_ds.withColumn('number_vmail_messages', col('number_vmail_messages').cast(IntegerType()))
transformed_ds = transformed_ds.withColumn('total_day_minutes', col('total_day_minutes').cast(DoubleType()))
transformed_ds = transformed_ds.withColumn('total_day_calls', col('total_day_calls').cast(IntegerType()))
transformed_ds = transformed_ds.withColumn('total_day_charge', col('total_day_charge').cast(DoubleType()))
transformed_ds = transformed_ds.withColumn('total_eve_minutes', col('total_eve_minutes').cast(DoubleType()))
transformed_ds = transformed_ds.withColumn('total_eve_calls', col('total_eve_calls').cast(IntegerType()))
transformed_ds = transformed_ds.withColumn('total_eve_charge', col('total_eve_charge').cast(DoubleType()))
transformed_ds = transformed_ds.withColumn('total_night_minutes', col('total_night_minutes').cast(DoubleType()))
transformed_ds = transformed_ds.withColumn('total_night_calls', col('total_night_calls').cast(IntegerType()))
transformed_ds = transformed_ds.withColumn('total_night_charge', col('total_night_charge').cast(DoubleType()))
transformed_ds = transformed_ds.withColumn('total_intl_minutes', col('total_intl_minutes').cast(DoubleType()))
transformed_ds = transformed_ds.withColumn('total_intl_calls', col('total_intl_calls').cast(IntegerType()))
transformed_ds = transformed_ds.withColumn('total_intl_charge', col('total_intl_charge').cast(DoubleType()))
transformed_ds = transformed_ds.withColumn('number_customer_service_calls', col('number_customer_service_calls').cast(IntegerType()))
transformed_ds = transformed_ds.withColumn('churn', col('churn').cast(IntegerType()))
transformed_ds = transformed_ds.withColumn('state_id', col('state_id').cast(IntegerType()))
transformed_ds = transformed_ds.withColumn('area_code_id', col('area_code_id').cast(IntegerType()))

In [13]:
transformed_ds

DataFrame[id: int, account_length: double, international_plan: int, voice_mail_plan: int, number_vmail_messages: int, total_day_minutes: double, total_day_calls: int, total_day_charge: double, total_eve_minutes: double, total_eve_calls: int, total_eve_charge: double, total_night_minutes: double, total_night_calls: int, total_night_charge: double, total_intl_minutes: double, total_intl_calls: int, total_intl_charge: double, number_customer_service_calls: int, churn: int, state_id: int, area_code_id: int]

In [14]:
# Check dataset TARGET variable balance
transformed_ds.groupBy('churn').count().show()

+-----+-----+
|churn|count|
+-----+-----+
|    1|  483|
|    0| 2850|
+-----+-----+



In [15]:
# The dataset is unbalanced
# First option is to increase the 'churn = 1 cases'

# Balance the dataset (increase churn=1 cases)
df_t0 = transformed_ds.where(col('churn') == 0)
df_t1 = transformed_ds.where(col('churn') == 1)

# Define the fraction
fraction = df_t0.count() / df_t1.count()

df_t1 = df_t1.sample(
    withReplacement = True, 
    fraction = fraction, 
    seed = 123)

# Combine majority class with upsampled minority class
df_balanced_1 = df_t0.union(df_t1)
 
# Display new class counts
df_balanced_1.groupBy('churn').count().show()

+-----+-----+
|churn|count|
+-----+-----+
|    1| 2861|
|    0| 2850|
+-----+-----+



In [16]:
# Save dataFrame to CSV file
df_balanced_1.toPandas().to_csv('../dataset/df_balanced_1.csv', index=False)

In [17]:
# The dataset is unbalanced
# Second option is to DECREASE the 'churn = 1 cases'

# Balance the dataset (increase churn=1 cases)
df_t0 = transformed_ds.where(col('churn') == 0)
df_t1 = transformed_ds.where(col('churn') == 1)

# Define the fraction
fraction = df_t1.count() / df_t0.count()

df_t0 = df_t0.sample(
    fraction = fraction, 
    seed = 123)

# Combine minority class with upsampled minority class
df_balanced_0 = df_t0.union(df_t1)
 
# Display new class counts
df_balanced_0.groupBy('churn').count().show()

+-----+-----+
|churn|count|
+-----+-----+
|    1|  483|
|    0|  500|
+-----+-----+



In [18]:
# Save dataFrame to CSV file
df_balanced_0.toPandas().to_csv('../dataset/df_balanced_0.csv', index=False)